In [4]:
import nltk
nltk.download('movie_reviews')
from nltk.corpus import movie_reviews

[nltk_data] Error loading movie_reviews: <urlopen error [WinError
[nltk_data]     10060] A connection attempt failed because the
[nltk_data]     connected party did not properly respond after a
[nltk_data]     period of time, or established connection failed
[nltk_data]     because connected host has failed to respond>


In [5]:
len(movie_reviews.fileids())

2000

In [6]:
# Accessing movie review of particular file
movie_reviews.words(movie_reviews.fileids()[1])

['the', 'happy', 'bastard', "'", 's', 'quick', 'movie', ...]

In [7]:
# Storing review with categories
documents=[]
for category in movie_reviews.categories():
    for fileid in movie_reviews.fileids(category):
        documents.append((movie_reviews.words(fileid),category))

In [8]:
import random
random.shuffle(documents)
documents[5][0]

['with', 'more', 'and', 'more', 'television', 'shows', ...]

In [9]:
# Adding stopwords and punctuations
import string
from nltk.corpus import stopwords 
nltk.download('stopwords')

punctuations=list(string.punctuation)
stops=(stopwords.words('english'))
stops=stops+punctuations


[nltk_data] Error loading stopwords: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


In [10]:
# New way of creating POS_TAG

from nltk.tag.api import TaggerI
from nltk.tag.util import str2tuple, tuple2str, untag
from nltk.tag.sequential import (
    SequentialBackoffTagger,
    ContextTagger,
    DefaultTagger,
    NgramTagger,
    UnigramTagger,
    BigramTagger,
    TrigramTagger,
    AffixTagger,
    RegexpTagger,
    ClassifierBasedTagger,
    ClassifierBasedPOSTagger,
)
from nltk.tag.brill import BrillTagger
from nltk.tag.brill_trainer import BrillTaggerTrainer
from nltk.tag.tnt import TnT
from nltk.tag.hunpos import HunposTagger
from nltk.tag.stanford import StanfordTagger, StanfordPOSTagger, StanfordNERTagger
from nltk.tag.hmm import HiddenMarkovModelTagger, HiddenMarkovModelTrainer
from nltk.tag.senna import SennaTagger, SennaChunkTagger, SennaNERTagger
from nltk.tag.mapping import tagset_mapping, map_tag
from nltk.tag.crf import CRFTagger
from nltk.tag.perceptron import PerceptronTagger

from nltk.data import load, find
import nltk
nltk.download('averaged_perceptron_tagger')
RUS_PICKLE = (
    "taggers/averaged_perceptron_tagger_ru/averaged_perceptron_tagger_ru.pickle"
)

[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [WinError 10060] A connection attempt failed because
[nltk_data]     the connected party did not properly respond after a
[nltk_data]     period of time, or established connection failed
[nltk_data]     because connected host has failed to respond>


In [11]:

def _get_tagger(lang):
    if lang == "rus":
        tagger = PerceptronTagger(False)
        ap_russian_model_loc = "file:" + str(find(RUS_PICKLE))
        tagger.load(ap_russian_model_loc)
    else:
        tagger = PerceptronTagger()
    return tagger

In [12]:

def _pos_tag(tokens, tagset=None, tagger=None, lang=None):
    # Currently only supports English and Russian.
    if lang not in ["eng", "rus"]:
        raise NotImplementedError(
            "Currently, NLTK pos_tag only supports English and Russian "
            "(i.e. lang='eng' or lang='rus')"
        )
    # Throws Error if tokens is of string type
    elif isinstance(tokens, str):
        raise TypeError("tokens: expected a list of strings, got a string")

    else:
        tagged_tokens = tagger.tag(tokens)
        if tagset:  # Maps to the specified tagset.
            if lang == "eng":
                tagged_tokens = [
                    (token, map_tag("en-ptb", tagset, tag))
                    for (token, tag) in tagged_tokens
                ]
            elif lang == "rus":
                # Note that the new Russian pos tags from the model contains suffixes,
                # see https://github.com/nltk/nltk/issues/2151#issuecomment-430709018
                tagged_tokens = [
                    (token, map_tag("ru-rnc-new", tagset, tag.partition("=")[0]))
                    for (token, tag) in tagged_tokens
                ]
        return tagged_tokens

In [13]:
def pos_tag(tokens, tagset=None, lang="eng"):
    tagger = _get_tagger(lang)
    return _pos_tag(tokens, tagset, tagger, lang)

In [14]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
# from nltk import pos_tag
nltk.download('wordnet')
from nltk.corpus import wordnet

# converting pos_tag to lemmatizer pos
def make_pos_simple(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN


[nltk_data] Error loading wordnet: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


In [15]:
#Main Function:- Data Cleaning-Removing Stop words and Lemmatisation
def remove_stop(sentence):
    output=[]
    for word in sentence:
        if word.lower() not in stops:
            # word should be passed as an array into pos tag
            pos=pos_tag([word])
            # array(word,pos)
            clean_word=lemmatizer.lemmatize(word,pos=make_pos_simple(pos[0][1]))
            output.append(clean_word.lower())
    return output



In [16]:
#Lammetizing top 10 documents
documents=[(remove_stop(document),category) for document,category in documents]

In [17]:
training_doc=documents[0:1500]
testing_doc=documents[1500:]

In [20]:
# building feature set
# taking the mostly repeated words as feature(frequency distribution)
all_words=[]
for doc in training_doc:
    all_words+=doc[0]
import nltk
freq=nltk.FreqDist(all_words)
common=freq.most_common(1000)
# Storing most common words as features 
features=[i[0] for i in common]
print(len(features))

1000


In [21]:
# now check if the most common features are present in particular document or not
def get_common(words):
    current_features={}
    words_set=set(words)
    for w in features:
        current_features[w]=w in words_set
    return current_features


In [22]:
training_data=[(get_common(doc),category) for doc,category in training_doc]
testing_data=[(get_common(doc),category) for doc,category in testing_doc]

In [23]:
# Using Naive Bayes Classifier to Train
from nltk import NaiveBayesClassifier
classifier=NaiveBayesClassifier.train(training_data)

In [24]:
nltk.classify.accuracy(classifier,testing_data)

0.742

In [25]:
classifier.show_most_informative_features(10)

Most Informative Features
                   waste = True              neg : pos    =      5.8 : 1.0
              ridiculous = True              neg : pos    =      5.6 : 1.0
                   awful = True              neg : pos    =      4.9 : 1.0
                  stupid = True              neg : pos    =      4.4 : 1.0
                  boring = True              neg : pos    =      4.0 : 1.0
                terrible = True              neg : pos    =      3.7 : 1.0
               realistic = True              pos : neg    =      3.6 : 1.0
                    mess = True              neg : pos    =      3.4 : 1.0
               excellent = True              pos : neg    =      3.4 : 1.0
               memorable = True              pos : neg    =      3.3 : 1.0
